In [1]:
%reload_ext watermark
%watermark -v -p tqdm,re,joblib,shutil,pandas,numpy,cv2,tensorflow,numpy,matplotlib,sklearn

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.34.0

tqdm      : 4.63.1
re        : 2.2.1
joblib    : 1.1.0
shutil    : unknown
pandas    : 1.4.1
numpy     : 1.22.3
cv2       : 4.6.0
tensorflow: 2.8.0
matplotlib: 3.5.1
sklearn   : 1.0.2



In [2]:
import glob,tqdm,re,os,datetime,joblib,time,shutil
import pandas as pd
import numpy as np

In [3]:
def merge_df(tsv_files):
    dfs=list()
    for tsv_file in tsv_files:
        df=pd.read_csv(tsv_file ,sep='\t')
        tsv_path=os.path.dirname(tsv_file)
        if len(re.compile("SN").findall(tsv_path))==1:
                bases=tsv_file.replace("anonymized","").replace('-','/').replace('_', '/').replace('.','/').split('/')
                base=[i for i in bases if re.compile('right|left|all').findall(i)]
        elif len(re.compile("vhit").findall(tsv_path))==1:
            bases=tsv_file.replace("all","").replace('-','/').replace('_', '/').replace('.','/').split('/')
            base=[i for i in bases if re.compile('Lateral|LARP|RALP|RPLA|Spon|Thrust|Straight').findall(i)]
            if len(base)!=1:
                base=[i for i in base if re.compile("Head Thrust 1").findall(i)]
                if len(base)==0:
                    base=[i for i in bases if re.compile('Lateral|LARP|RALP|RPLA|Spon|Thrust|Straight').findall(i)]
                    base=[i for i in base if re.compile("Head Thrust 0").findall(i)]
        
        if len(base)!=1:
            # raise
            base=['']
        df['data_type']=base[0]
        file_name=tsv_file.split('/')[-1].split('.')[0].split('-')
        df['file_name']='-'.join(tsv_file.split('/')[-1].split('.')[0].split('-')[:-1])

        if (len([i for i in file_name if re.compile('Nystagmus').findall(i)])!=0)&(df.shape[0]!=0):
            df['file_name']=df['file_name']+'-'+[i for i in file_name if re.compile('left|right|all').findall(i)][0]
        dfs.append(df)
        
    df=None
    if len(dfs)==0:
        raise
    df=pd.concat(dfs,axis=0,ignore_index=True)
    return df

In [4]:
def check_img(img_df,test_path,model_type):
    matchs=list()
    if img_df.shape[0]!=0:
        idx=list()
        for frame, data_type, file_name in img_df[["frame_number","data_type","file_name"]].values:
            file_name=file_name.replace("--right",'');file_name=file_name.replace("--left",'')
            file_name=file_name.replace("--all",'')
            test_img_path=test_path.replace("2.라벨링","1.원천")
            
            match=glob.glob(f"{test_img_path}/blink/*{data_type}*frame_{frame}-*.png")
            if len(match)==0:
                match=glob.glob(f"{test_img_path}/blink/*frame_{frame}-*{data_type}.png")
            if len(match)>1:
                match=glob.glob(f"{test_img_path}/blink/{file_name}-frame_{frame}-*.png")
                if len(match)==0:
                    match=glob.glob(f"{test_img_path}/blink/{file_name}-frame_{frame}-*{data_type}.png")
            if len(match)!=1:
                # do not find png
                new_dir=test_path.replace(f"/home/data/{model_type}/2.라벨링데이터",'/lustre/data/vHIT/s3/upload/20221129_update/20221207/merge')
                match=glob.glob(f"{new_dir}/blink/*{data_type}*frame_{frame}-*.png")
                if len(match)==0:
                    match=glob.glob(f"{new_dir}/blink/*frame_{frame}-*{data_type}.png")
                if len(match)>1:
                    match=glob.glob(f"{new_dir}/blink/{file_name}-frame_{frame}-*.png")+\
                        glob.glob(f"{new_dir}/blink/{file_name}--frame_{frame}-*.png")
                    if len(match)==0:
                        match=glob.glob(f"{new_dir}/blink/{file_name}-frame_{frame}-*{data_type}.png")
                elif len(match)==0:
                    match=glob.glob(f"{new_dir}/blink/{file_name}-frame_{frame}-*.png")+\
                        glob.glob(f"{new_dir}/blink/{file_name}--frame_{frame}-*.png")
                if len(match)!=1:
                    raise
            matchs.append(match[0])
    return matchs

In [5]:
def check_video(video_df, test_path,model_type):
    matchs=list()
    if video_df.shape[0]!=0:
        idx=list()
        for frame,data_type,file_name in video_df[["frame_number","data_type",'file_name']].values:
            file_name=file_name.replace("--right",'');file_name=file_name.replace("--left",'')
            file_name=file_name.replace("--all",'')
            test_img_path=test_path.replace("2.라벨링","1.원천")
            
            
            match=glob.glob(f"{test_img_path}/horizontal_vertical/*{data_type}*frame_{frame}-*.avi")
            if len(match)==0:
                match=glob.glob(f"{test_img_path}/horizontal_vertical/*frame_{frame}-*{data_type}.avi")
            if len(match)>1:
                match=glob.glob(f"{test_img_path}/horizontal_vertical/{file_name}-frame_{frame}-*.avi")
                if len(match)==0:
                    match=glob.glob(f"{test_img_path}/horizontal_vertical/{file_name}-frame_{frame}-*{data_type}.avi")
            if len(match)!=1:
                # raise
                new_dir=test_path.replace(f"/home/data/{model_type}/2.라벨링데이터",'/lustre/data/vHIT/s3/upload/20221129_update/20221207/merge')
                match=glob.glob(f"{new_dir}/horizontal_vertical/*{data_type}*frame_{frame}-*.avi")
                if len(match)==0:
                    match=glob.glob(f"{new_dir}/horizontal_vertical/*frame_{frame}-*{data_type}.avi")
                if len(match)>1:
                    match=glob.glob(f"{new_dir}/horizontal_vertical/{file_name}-frame_{frame}-*.avi")+\
                        glob.glob(f"{new_dir}/horizontal_vertical/{file_name}--frame_{frame}-*.avi")
                    if len(match)==0:
                        match=glob.glob(f"{new_dir}/horizontal_vertical/{file_name}-frame_{frame}-*{data_type}.avi")
                elif len(match)==0:
                    match=glob.glob(f"{new_dir}/horizontal_vertical/{file_name}-frame_{frame}-*.avi")+\
                        glob.glob(f"{new_dir}/horizontal_vertical/{file_name}--frame_{frame}-*.avi")
                if len(match)!=1:
                    raise
            matchs.append(match[0])
    return matchs

In [6]:
def f_img(test_path,model_type):
    tsv_files=glob.glob(f"{test_path}/*tsv")
    df=merge_df(tsv_files)    
    img_df=df.loc[df.blink=='C'].copy()
    img_df.reset_index(drop=True)
    img_df.loc[:,'file_path']=check_img(img_df,test_path,model_type)
    img_df=img_df.loc[:,['blink_tagging','file_path']]
    return img_df

In [7]:
def f_video(test_path,model_type):
    tsv_files=glob.glob(f"{test_path}/*tsv")
    df=merge_df(tsv_files)
    video_df=df[df.blink=='O'].copy()
    video_df.reset_index(drop=True)
    video_df.loc[:,'file_path']=check_video(video_df,test_path,model_type)
    video_df=video_df.loc[:,['horizontal','file_path']]
    return video_df

In [8]:
"""
/home/data/train/2.라벨링데이터/Central/NE_C_00000/vhit
"""
test_paths=sorted(glob.glob("/home/data/train/2.라벨링데이터/**/**/**"))
print(datetime.datetime.today())

2022-12-16 03:36:24.211984


In [9]:
if not os.path.exists("눈깜빡임분류_train.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_img)(i,"train") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("눈깜빡임분류_train.tsv",index=False,sep='\t')

100%|██████████| 14705/14705 [00:47<00:00, 310.67it/s]


In [10]:
if not os.path.exists("안진방향판별_train.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_video)(i,"train") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("안진방향판별_train.tsv",index=False,sep='\t')

100%|██████████| 14705/14705 [00:24<00:00, 606.11it/s]


In [11]:
test_paths=sorted(glob.glob("/home/data/test/2.라벨링데이터/**/**/**"))
print(datetime.datetime.today())

2022-12-16 03:39:30.561260


In [12]:
if not os.path.exists("눈깜빡임분류_test.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_img)(i,"test") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("눈깜빡임분류_test.tsv",index=False,sep='\t')

100%|██████████| 2954/2954 [00:05<00:00, 558.25it/s]


In [13]:
if not os.path.exists("안진방향판별_test.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_video)(i,"test") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("안진방향판별_test.tsv",index=False,sep='\t')

100%|██████████| 2954/2954 [00:04<00:00, 667.52it/s]


In [14]:
test_paths=sorted(glob.glob("/home/data/valid/2.라벨링데이터/**/**/**"))
print(datetime.datetime.today())

2022-12-16 03:39:53.067548


In [15]:
if not os.path.exists("눈깜빡임분류_valid.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_img)(i,"valid") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("눈깜빡임분류_valid.tsv",index=False,sep='\t')

100%|██████████| 2989/2989 [00:05<00:00, 498.90it/s]


In [16]:
if not os.path.exists("안진방향판별_valid.tsv"):
    with joblib.Parallel(n_jobs=-3) as parallel:
        results = parallel(joblib.delayed(f_video)(i,"valid") for i in tqdm.tqdm(test_paths[:]))
    pd.concat(results).to_csv("안진방향판별_valid.tsv",index=False,sep='\t')

100%|██████████| 2989/2989 [00:04<00:00, 684.30it/s]


In [17]:
print(datetime.datetime.today())

2022-12-16 03:40:10.631850
